# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [4]:
url = "https://books.toscrape.com/catalogue/page-1.html"
response = requests.get(url)
response

<Response [200]>

In [5]:
soup = BeautifulSoup(response.content)

In [6]:
books_grid = soup.find("ol", attrs = {"class":"row"})

In [7]:
books = books_grid.find_all("li", attrs = {"class":"col-xs-6 col-sm-4 col-md-3 col-lg-3"})

In [ ]:
BASE_URL = "http://books.toscrape.com/"

# Map star-rating class names to numeric values
RATING_MAP = {
    "One": 1,
    "Two": 2,
    "Three": 3,
    "Four": 4,
    "Five": 5
}

def get_full_url(relative_url):
    return BASE_URL + relative_url.replace("../", "").replace("catalogue/", "")

def scrape_book_detail(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    title = soup.h1.text.strip()

    description_tag = soup.select_one("#product_description ~ p")
    description = description_tag.text.strip() if description_tag else "No description"

    table = soup.select("table.table.table-striped tr")
    upc = table[0].td.text.strip() if len(table) > 0 else "N/A"
    availability = table[5].td.text.strip() if len(table) > 5 else "N/A"

    breadcrumb = soup.select("ul.breadcrumb li a")
    genre = breadcrumb[2].text.strip() if len(breadcrumb) > 2 else "Unknown"

    return {
        "UPC": upc,
        "Title": title,
        "Genre": genre,
        "Availability": availability,
        "Description": description
    }

def scrape_books(min_rating=4, max_price=20):
    """Scrape books from the site matching rating and price filters."""
    all_books = []
    page_url = "catalogue/page-1.html"

    while page_url:
        response = requests.get(BASE_URL + page_url)
        soup = BeautifulSoup(response.content, "html.parser")
        book_items = soup.select("article.product_pod")

        for book in book_items:
            # Extract rating
            rating_class = book.p['class'][1]
            rating = RATING_MAP.get(rating_class, 0)

            # Extract price
            price_text = book.select_one("p.price_color").text.strip().replace("£", "")
            price = float(price_text)

            if rating >= min_rating and price <= max_price:
                # Link to book detail page
                detail_href = book.h3.a['href']
                detail_url = get_full_url(detail_href)

                # Get detailed book data
                book_data = scrape_book_detail(detail_url)

                # Add price and rating to book_data
                book_data.update({
                    "Price (£)": price,
                    "Rating": rating
                })

                all_books.append(book_data)

        # Go to next page if available
        next_button = soup.select_one("li.next > a")
        if next_button:
            next_href = next_button['href']
            page_url = "catalogue/" + next_href
        else:
            page_url = None


    return pd.DataFrame(all_books)

if __name__ == "__main__":
    # Example: get books with rating >= 4 and price <= 20
    df = scrape_books(min_rating=4, max_price=20)
    print(df.head())
    df.to_csv("filtered_books.csv", index=False)
    print("\nScraping complete. Results saved to 'filtered_books.csv'.")


   UPC          Title    Genre Availability     Description  Price (£)  Rating
0  N/A  404 Not Found  Unknown          N/A  No description      17.46       5
1  N/A  404 Not Found  Unknown          N/A  No description      17.66       5
2  N/A  404 Not Found  Unknown          N/A  No description      15.94       5
3  N/A  404 Not Found  Unknown          N/A  No description      14.27       4
4  N/A  404 Not Found  Unknown          N/A  No description      19.49       4

Scraping complete. Results saved to 'filtered_books.csv'.
